In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "urvishp80"
os.environ['KAGGLE_KEY'] = "efaeb62d2fd8a94891f65cde49c5d924"

In [ ]:
!kaggle datasets download -d lizhecheng/llama2-7b-hf
!unzip llama2-7b-hf.zip -d llama2-7b-hf

100% 9.66G/9.66G [04:09<00:00, 43.5MB/s]
100% 9.66G/9.66G [04:09<00:00, 41.6MB/s]
Archive:  llama2-7b-hf.zip
  inflating: llama2-7b-hf/Llama2-7b-hf/config.json  
  inflating: llama2-7b-hf/Llama2-7b-hf/generation_config.json  
  inflating: llama2-7b-hf/Llama2-7b-hf/pytorch_model-00001-of-00002.bin  
  inflating: llama2-7b-hf/Llama2-7b-hf/pytorch_model-00002-of-00002.bin  
  inflating: llama2-7b-hf/Llama2-7b-hf/pytorch_model.bin.index.json  
  inflating: llama2-7b-hf/Llama2-7b-hf/special_tokens_map.json  
  inflating: llama2-7b-hf/Llama2-7b-hf/tokenizer.json  
  inflating: llama2-7b-hf/Llama2-7b-hf/tokenizer.model  
  inflating: llama2-7b-hf/Llama2-7b-hf/tokenizer_config.json  


In [ ]:
!pip install datasets -q
!pip install peft -q
!pip install transformers -q
!pip install trl -q
!pip install bitsandbytes -q
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.5 MB/s eta 0:00:00


In [ ]:
# using single gpu for this case

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
import numpy as np
from dataclasses import dataclass, field
from typing import Optional
import torch

from datasets import load_dataset, Dataset
from peft import (LoraConfig, prepare_model_for_kbit_training, get_peft_model,
                  PeftModel, PeftConfig, TaskType, PeftModelForSequenceClassification)
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSequenceClassification, AutoModelForCausalLM, AutoModel,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoModelForMultipleChoice
)
from trl import RewardTrainer, SFTTrainer, DataCollatorForCompletionOnlyLM

import random
import os
import gc
import pandas as pd
pd.set_option('display.max_colwidth', None)
tqdm.pandas()

In [ ]:
class CFG:
    base_model = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-large"
    seed = 42

In [ ]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state


random_state = set_seed(CFG.seed)

### Load Model and Tokenizer

In [ ]:
# change model_name to the model of your choice.
# This can be either name of the model on huggingface (requires internet) or path to the model
model_name = "/content/llama2-7b-hf/Llama2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtyp=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
# this should be set as False for finetuning
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Preprocess Data

In [ ]:
df = pd.read_csv("/content/data.csv")
df.head()

text
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  <s><<SYS>>\nYou are an intelligent assistant.\n<</SYS>>\n\n</s> [INST] ### Input: Suppose you are a programmer and you are enriched by programming knowledge. You will be going through other programmers mail sent to you and you will be extracting all the important information out of the mail and composing a blog post. Even if the mail is divided into parts and parts, your extraction summary should not be in bullet points. It should be in multiple paragraphs. I repeat, never in bullet points. You have to follow some rules while giving a detailed summary. \n                    The rules are below:\n                        1. While extracting, avoid using phrases referring to the context. Instead, directly present the information or points covered.  Do not introduce sentences with phrases like: "The context discusses...", "In this context..." or "The context covers..." or "The context questions..." etc\n                        2. The summary tone should be formal and full of information.\n                        3. Add spaces after using punctuation and follow all the grammatical rules.\n                        4. Try to retain all the links provided and use them in proper manner at proper place.\n                        5. The farewell part of the email should be completely ignored.\n                        6. Ensure that summary is not simply a rephrase of the original content with minor word changes, but a restructured and simplified rendition of the main points.\n                        7. Most importantly, this extracted information should be relative of the size of the email. If it is a bigger email, the extracted summary can be longer than a very short email. \n                        Context: Maybe the anonymous credentials approach from [2] could use credentials loaded with sats?\n\n\n                        ### Output: [/INST] In a recent email, the sender mentioned the possibility of using anonymous credentials loaded with sats as part of the anonymous credentials approach discussed in reference [2]. The idea is to leverage the use of sats (short for satoshis) within the context of anonymous credentials. While the email does not provide further details or explanations, it raises an interesting concept that could potentially enhance the functionality and security of anonymous credentials.Anonymous credentials are a cryptographic tool used to prove certain statements about oneself without revealing any additional information. They enable individuals to authenticate themselves or assert specific attributes without compromising their privacy. The approach proposed in reference [2] explores the use of anonymous credentials in a broader context, possibly in relation to blockchain technology and decentralized s

In [ ]:
train_dataset = Dataset.from_pandas(df)
train_dataset[0]

{'text': '<s><<SYS>>\nYou are an intelligent assistant.\n<</SYS>>\n\n</s> [INST] ### Input: Suppose you are a programmer and you are enriched by programming knowledge. You will be going through other programmers mail sent to you and you will be extracting all the important information out of the mail and composing a blog post. Even if the mail is divided into parts and parts, your extraction summary should not be in bullet points. It should be in multiple paragraphs. I repeat, never in bullet points. You have to follow some rules while giving a detailed summary. \n                    The rules are below:\n                        1. While extracting, avoid using phrases referring to the context. Instead, directly present the information or points covered.  Do not introduce sentences with phrases like: "The context discusses...", "In this context..." or "The context covers..." or "The context questions..." etc\n                        2. The summary tone should be formal and full of info

### Prepare Model

In [ ]:
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [ ]:
# for llama 2 (they need different target module)
qlora_config = LoraConfig(
    r=1,
    lora_alpha=4,  # parameter for scaling
    target_modules=['up_proj', 'o_proj', 'q_proj', 'k_proj', 'v_proj', 'gate_proj', 'down_proj'],
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head"]
)

# # Change the LORA hyperparameters accordingly to fit your use case
# peft_config = LoraConfig(
#     r=128,
#     lora_alpha=16,
#     target_modules=find_all_linear_names(base_model),
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     )

base_model = get_peft_model(base_model, peft_config)

trainable params: 264,642,560 || all params: 6,871,986,176 || trainable%: 3.851034522220785


In [ ]:
training_args = TrainingArguments(
    output_dir='SFT-llama2-7b',
    overwrite_output_dir = True,
    warmup_ratio=0.1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    remove_unused_columns=False,
    optim="paged_adamw_8bit",
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
    load_best_model_at_end=True,
    save_total_limit = 1,
    fp16=False,
    bf16=True,
    weight_decay=0.01,
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
    peft_config=qlora_config,
    dataset_text_field="text",
    max_seq_length=3000,
    data_collator=DataCollatorForCompletionOnlyLM(tokenizer=tokenizer,
                                                  response_template="### Output: ")
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.config.use_cache = False
trainer.train()
trainer.save_model('llama-v2-fine-tuned')

del model

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from datetime import datetime
datetime.now()